Task 3: Organizing PDF Files for Companies

The company folders should be in the same directory where the script is run from

In [266]:
import os
import re
import shutil
from PyPDF2 import PdfReader
from datetime import datetime

In [267]:
pdf_pattern = r'[\w\d\s]+\.pdf'

cwd = os.getcwd()

apple_docs_folder = f'{cwd}{os.sep}Apple docs'
tesla_docs_folder = f'{cwd}{os.sep}Tesla docs'

apple_listed_dir = ','.join(os.listdir(apple_docs_folder))
tesla_listed_dir = ','.join(os.listdir(tesla_docs_folder))

apple_new_pdfs = re.findall(pdf_pattern, apple_listed_dir)
tesla_new_pdfs = re.findall(pdf_pattern, tesla_listed_dir)

Extract the date from each of the new document’s titles

In [268]:
def read_pdf_first_page(pdf_file_name):
    with open(pdf_file_name, 'rb') as opened_file:
        reader = PdfReader(opened_file)
        
        first_page = reader.pages[0]
        first_page_data = first_page.extract_text()
        
    return first_page_data


def get_pdf_file_dates(pdf_file, company_folder):
    date_pattern = r'[A-Z][a-z]+[ ]+\d+,[ ]+\d{4}'
    file_path = f'{company_folder}{os.sep}{pdf_file}'
    
    first_page_data = read_pdf_first_page(file_path)    
    dates = re.findall(date_pattern, first_page_data)
    date = dates.pop(0) if dates else None

    return date

In [269]:
pdf_extracted_dates = {
    'apple docs': {pdf: get_pdf_file_dates(pdf, apple_docs_folder) for pdf in apple_new_pdfs}, 
    'tesla docs': {pdf: get_pdf_file_dates(pdf, tesla_docs_folder) for pdf in tesla_new_pdfs}
}

Move the PDFs to the appropriate folders corresponding to the correct quarters
Bonus: Categorize the new documents

In [271]:
quarter_dates_ranges = {
    'Q1': [(1, 1), (3, 31)],
    'Q2': [(4, 1), (6, 30)],
    'Q3': [(7, 1), (9, 30)],
    'Q4': [(10, 1), (12, 31)],
}

In [272]:
def categorize_pdf_files(date):
    for dates in quarter_dates_ranges.values():
        starting_date = dates[0]
        ending_date = dates[1]
        mid_of_quarter = tuple(map(sum, zip(starting_date, (1, 14))))
        
        if starting_date <= date <= mid_of_quarter:
            return 'Cat1'
        elif mid_of_quarter < date <= ending_date:
            return 'Cat2'

In [278]:
def format_extracted_date(date):
    day = int(date.replace(',', '').split()[1])
    month = datetime.strptime(date.split()[0], "%B").month
    return month, day
    

def get_quarter(date):    
    for quarter, dates_list in quarter_dates_ranges.items():
        starting_date = dates_list[0]
        ending_date = dates_list[1]
        
        if starting_date <= date <= ending_date:
            return quarter
        

def check_quarter_directory_exists(quarter, company_folder):
    quarter_path = os.path.join(company_folder, quarter)
    if not os.path.exists(quarter_path):
        os.makedirs(quarter_path)    
    

def move_pdf_to_quarter_folder(pdf_file, date_extracted, company_folder):
    date = format_extracted_date(date_extracted)
    quarter = get_quarter(date)
    category = categorize_pdf_files(date)
    
    destination_path = f'{company_folder}{os.sep}{quarter}{os.sep}{category}.pdf'
    pdf_abs_path = f'{company_folder}{os.sep}{pdf_file}'
    
    check_quarter_directory_exists(quarter, company_folder)

    shutil.move(pdf_abs_path, destination_path)


def organize_new_company_pdfs(company_name):
    company_folder = apple_docs_folder if company_name in apple_docs_folder else tesla_docs_folder
    company_docs = f'{company_name.lower()} docs'
    pdfs = pdf_extracted_dates.get(company_docs)

    for pdf_file, extracted_date in pdfs.items():
        if extracted_date:
            move_pdf_to_quarter_folder(pdf_file, extracted_date, company_folder)
                  

In [280]:
organize_new_company_pdfs('Apple')
organize_new_company_pdfs('Tesla')